# Formação Cientista de Dados - Data Science Academy

# **Projeto com feedback 7 - Medicina Personalizada - Redefinindo o Tratamento de Câncer**

**Nome**: Rafael Henrique Gallo

**Descrição**: Muito tem sido dito durante os últimos anos sobre como a medicina de
precisão e, mais concretamente, como o teste genético, vai provocar disrupção no
tratamento de doenças como o câncer.
Mas isso ainda está acontecendo apenas parcialmente devido à enorme
quantidade de trabalho manual ainda necessário. Neste projeto, tentaremos levar
a medicina personalizada ao seu potencial máximo.
Uma vez sequenciado, um tumor cancerígeno pode ter milhares de
mutações genéticas. O desafio é distinguir as mutações que contribuem para o
crescimento do tumor das mutações.
Atualmente, esta interpretação de mutações genéticas está sendo feita
manualmente. Esta é uma tarefa muito demorada, onde um patologista clínico tem
que revisar manualmente e classificar cada mutação genética com base em
evidências da literatura clínica baseada em texto.
Para este projeto, o MSKCC (Memorial Sloan Kettering Cancer Center) está
disponibilizando uma base de conhecimento anotada por especialistas, onde
pesquisadores e oncologistas de nível mundial anotaram manualmente milhares
de mutações.

**Objetivo problema**: Neste projeto, você vai desenvolver um algoritmo de Aprendizado de Máquina que, usando essa base de conhecimento como uma linha de base, classifica automaticamente as variações genéticas.

- Recomendamos o uso de Python para criação de um modelo de Deep Learning com Keras.

**O dataset completo pode ser encontrado em**:
https://www.kaggle.com/c/msk-redefining-cancer-treatment/data

# **Descrição projeto**

Nesse projeto, desenvolvi uma rede neural com o objetivo de classificar as variações genéticas. Utilizei modelos de aprendizado de máquina para processamento de linguagem natural, como o Word2Vec e o Doc2Vec, para extrair informações relevantes das palavras-chave. Em seguida, utilizei o modelo de extração de tópicos LDA para identificar e extrair as palavras-chave pertinentes. Por fim, implementei uma rede neural para classificar as variações genéticas.

O modelo Word2Vec é uma técnica popular que mapeia palavras em vetores de alta dimensionalidade, capturando relações semânticas e sintáticas entre elas. Essa abordagem permitiu que a rede neural obtivesse uma compreensão mais profunda do significado das palavras relacionadas às variações genéticas.

O Doc2Vec, por sua vez, expande o conceito do Word2Vec para documentos inteiros, atribuindo vetores a cada documento. Essa técnica possibilitou a extração de informações contextuais relevantes das variações genéticas, considerando o texto completo em que elas estão inseridas.

Além disso, apliquei o modelo de extração de tópicos LDA (Latent Dirichlet Allocation) para identificar os tópicos principais presentes nos documentos relacionados às variações genéticas. Essa abordagem permitiu a obtenção de palavras-chave específicas de cada tópico, fornecendo uma visão mais abrangente e organizada das informações relevantes.

Por fim, desenvolvi uma rede neural para classificar as variações genéticas com base nas informações extraídas pelos modelos anteriores. Essa rede neural foi treinada utilizando conjuntos de dados rotulados, nos quais cada variação genética estava associada a uma classe específica. Com a rede neural treinada, foi possível realizar classificações precisas de novas variações genéticas com base nos padrões aprendidos durante o treinamento.

Em suma, por meio da combinação dos modelos de processamento de linguagem natural Word2Vec, Doc2Vec e extração de tópicos LDA, juntamente com uma rede neural para classificação, obtive um sistema eficiente para a classificação de variações genéticas. Essa abordagem possibilitou a análise e compreensão mais precisa das informações genéticas, contribuindo para avanços na área da genômica e sua aplicação em diversas áreas da saúde e biologia.

In [ ]:
# Baixando pacotes necesarios
!pip install watermark
!pip install Keras-Preprocessing

In [ ]:
#Importação das bibliotecas

# Bibliotecas sistema
import os
import re
import tqdm
import string
import unicodedata
import itertools

# Biblioteca para manipulação de arquivos
import pandas as pd
import numpy as np
from numpy.linalg import lstsq
from numpy.random import randn, seed

# Bibliotecas de visualização
import seaborn as sns
import matplotlib as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import wordcloud
from wordcloud import WordCloud

# Bibliotecas rede neural
import tensorflow
import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.utils import class_weight

# Bibliotecas para extração tópicos
from sklearn.decomposition import LatentDirichletAllocation

# Vetorização palavras
from sklearn.feature_extraction.text import CountVectorizer

# Importando NLTK NLP
# Baixando pacote do nltk
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import conll2000
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk.chunk import ne_chunk
from nltk import pos_tag

# Baixando todos os complementos NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

# Versão do python
from platform import python_version
print('Versão Jupyter Notebook neste projeto:', python_version())

# Carregar as versões das bibliotecas
import watermark

# Warnings retirar alertas
import warnings
warnings.filterwarnings("ignore")

# Configuração fundo dos gráficos e estilo, tamanho da fonte
sns.set_style('whitegrid')
pd.options.display.float_format = '{:,.2f}'.format
seed(42)

# Verficações da versões das bibliotecas
%reload_ext watermark
%watermark -a "Rafael Gallo" --iversions

# **Base dados**

In [ ]:
# Tabelas
data_train = pd.read_csv('training_variants')
test_data = pd.read_csv('test_variants')
data_train_text = pd.read_csv('training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
text_data = pd.read_csv('test_text', sep='\|\|', engine='python', header=None, skiprows=1, names=["ID","Text"])
data1 = pd.read_csv('stage1_solution_filtered.csv')

# Merge tabelas
data = pd.merge(data_train, data_train_text, how='left', on='ID').fillna('')
train = pd.merge(data_train, data_train_text, how='left', on='ID').fillna('')
train = pd.merge(data_train, data_train_text, how='left', on='ID').fillna('')
test = pd.merge(test_data, text_data, how='left', on='ID').fillna('')

# Trasformando tabela para númerica
data1['Class'] = pd.to_numeric(data1.drop('ID', axis=1).idxmax(axis=1).str[5:]).fillna(0).astype(np.int64)

# Index na tabela
test = test[test.index.isin(data1['ID'])]

# Merge nova tabela
df = test.merge(data1[['ID', 'Class']], on='ID', how='left')
df.head()

In [ ]:
# Visualizando os 5 últimos dados
df.tail()

In [ ]:
# Info dos dados
df.info()

In [ ]:
# Tipos dos dados
df.dtypes

In [ ]:
plt.figure()
ax = df['Gene'].value_counts().plot(kind='area')

ax.get_xaxis().set_ticks([])
ax.set_title('Gene Frequency Plot')
ax.set_xlabel('Gene')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
ax = df['Class'].value_counts().plot(kind='bar')

ax.set_title('Class Distribution Over Entries')
ax.set_xlabel('Class')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
%%time

# Gráfico de nuvem palavras mais faladas com dataset limpo
todos_palavras2 = ' '.join([message for message in df["Text"]])
nuvem_palavras2 = WordCloud(width = 900,
                           height = 600,
                           background_color="white",
                           collocations = True,
                           max_font_size = 200).generate(todos_palavras2)

plt.figure(figsize= (18.5, 25))
plt.title("Nuvem palavras limpo")
plt.ylabel("Total")
plt.xlabel("Nuvem")
plt.imshow(nuvem_palavras2, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
%%time

# Gráfico de nuvem palavras mais faladas com dataset limpo
todos_palavras2 = ' '.join([message for message in df["Text"]])
nuvem_palavras2 = WordCloud(width = 900,
                           height = 600,
                           background_color="black",
                           collocations = True,
                           max_font_size = 200).generate(todos_palavras2)

plt.figure(figsize= (18.5, 25))
plt.title("Nuvem palavras limpo")
plt.ylabel("Total")
plt.xlabel("Nuvem")
plt.imshow(nuvem_palavras2, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
%%time

# Gráfico de nuvem palavras mais faladas com dataset limpo
todos_palavras2 = ' '.join([message for message in df["Variation"]])
nuvem_palavras2 = WordCloud(width = 900,
                           height = 600,
                           background_color="white",
                           collocations = True,
                           max_font_size = 200).generate(todos_palavras2)

plt.figure(figsize= (18.5, 25))
plt.title("Nuvem palavras - Variation")
plt.ylabel("Total")
plt.xlabel("Nuvem")
plt.imshow(nuvem_palavras2, interpolation = 'bilinear')
plt.axis("off")
plt.show()

# **Pré-processamento**

In [ ]:
# Pré-processamento dos dados, se necessário
# Exemplo: converter para minúsculas e dividir o texto em palavras
df['Text'] = df['Text'].str.lower()
df['Text'] = df['Text'].str.split()

In [ ]:
# Crie uma lista de listas de tokens
sentences = df['Text'].tolist()

# **Modelo Word2Vec**

- Word2Vec é um modelo de processamento de linguagem natural (NLP) amplamente utilizado para representar palavras em forma de vetores. Ele foi desenvolvido por Mikolov et al. em 2013 como uma técnica eficaz para aprender representações de palavras em um espaço vetorial contínuo.


- A principal ideia por trás do Word2Vec é capturar o significado e as relações entre as palavras com base no contexto em que elas ocorrem. O modelo é treinado em grandes quantidades de texto não rotulado, como coleções de documentos ou até mesmo a internet, para aprender representações distribuídas de palavras. O Word2Vec utiliza uma abordagem de aprendizado não supervisionado, na qual palavras que aparecem em contextos similares tendem a ter representações vetoriais semelhantes. O modelo faz isso ao prever a probabilidade de ocorrência de uma palavra em relação às palavras próximas a ela em um determinado contexto. Essa previsão é realizada por meio de duas arquiteturas principais: Skip-gram e CBOW (Continuous Bag-of-Words).


- No modelo Skip-gram, a palavra central é usada para prever as palavras do contexto. Por exemplo, dada a frase "o gato está no telhado", o modelo Skip-gram tentaria prever a palavra "gato" com base nas palavras "o", "está", "no" e "telhado". Já no modelo CBOW, as palavras do contexto são usadas para prever a palavra central. Uma vez treinado, o modelo Word2Vec gera vetores de palavras em um espaço de alta dimensionalidade, em que palavras semanticamente relacionadas estão próximas umas das outras. Esses vetores podem ser usados como recursos para uma variedade de tarefas de processamento de linguagem natural, como classificação de texto, agrupamento de documentos, tradução automática, entre outros.



In [ ]:
# Treine o modelo Word2Vec

# Importando biblioteca
from gensim.models import Word2Vec

# Criando modelo
model = Word2Vec(sentences,
                 window=5,
                 min_count=5,
                 workers=4)

In [ ]:
# Salve o modelo treinado
model.save("modelo_word2vec.bin")

In [ ]:
# Carregue o modelo treinado
model = Word2Vec.load("modelo_word2vec.bin")

# Obtenha o vetor de uma palavra específica
word_vector = model.wv['cancer']
print("Vetor da palavra 'cancer':", word_vector)

In [ ]:
# Encontre as palavras mais similares a uma palavra específica
similar_words = model.wv.most_similar('treatment')
print("Palavras similares a 'treatment':", similar_words)

In [ ]:
# Realize operações com palavras
result = model.wv.most_similar(positive=['gene', 'mutation'],
                               negative=['cancer'])
print("Palavra mais similar a 'gene' e 'mutation', menos similar a 'cancer':", result)

# **Doc2Vec**

- Doc2Vec, também conhecido como Paragraph Vector, é um modelo de processamento de linguagem natural (NLP) que estende o conceito do Word2Vec para a representação de documentos inteiros em forma de vetores. Foi proposto por Le and Mikolov em 2014 como uma abordagem para capturar o contexto e o significado dos documentos de maneira distribuída. Assim como o Word2Vec, o Doc2Vec é baseado em aprendizado não supervisionado e utiliza redes neurais para aprender representações vetoriais de documentos. A ideia principal é atribuir um vetor único a cada documento, permitindo que ele seja comparado e relacionado a outros documentos de forma eficiente.

- O modelo Doc2Vec pode ser implementado de duas maneiras principais: PV-DM (Distributed Memory Paragraph Vector) e PV-DBOW (Distributed Bag-of-Words Paragraph Vector). No PV-DM, o modelo tenta prever a próxima palavra em um contexto, considerando tanto as palavras do documento quanto o próprio vetor do documento. Ele captura tanto o contexto local (palavras próximas) quanto o contexto global (o documento em si) durante o treinamento.

- No PV-DBOW, o modelo ignora as palavras do documento e concentra-se apenas em prever as palavras com base no vetor do documento. Dessa forma, o modelo aprende a representação vetorial do documento sem levar em consideração as palavras específicas nele contidas. Após o treinamento, cada documento é representado por um vetor de alta dimensionalidade no espaço vetorial. Esses vetores capturam informações contextuais e semânticas do documento, permitindo comparações e análises. O Doc2Vec é amplamente utilizado em várias aplicações de NLP, como classificação de documentos, agrupamento de documentos semelhantes, recuperação de informações, análise de sentimentos e muito mais. Ele permite que os modelos entendam e comparem documentos inteiros, em vez de tratar cada documento como uma coleção independente de palavras.

- Essa técnica é especialmente útil em cenários onde o contexto global dos documentos é importante, como na análise de textos longos, artigos científicos, resenhas de produtos, entre outros.

**Em resumo, o Doc2Vec é um modelo de NLP que aprende representações vetoriais de documentos inteiros, permitindo uma compreensão mais abrangente e contextual dos textos. Ele é uma extensão poderosa do Word2Vec e tem sido amplamente utilizado para uma variedade de tarefas e aplicações relacionadas ao processamento de linguagem natural**

In [ ]:
# Importando doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

# Carregar os dados
train_variants = pd.read_csv("/content/training_variants")
training_text = pd.read_csv('training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

In [ ]:
# Pré-processamento dos dados
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = simple_preprocess(text)
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# Criar os documentos
documents = []
for idx, row in training_text.iterrows():
    doc_id = row['ID']
    text = row['Text']
    tokens = preprocess_text(text)
    documents.append(TaggedDocument(tokens, [doc_id]))

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

# Treinar o modelo Doc2Vec
vector_size = 100
window_size = 5
epochs = 20

# Modelo
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                epochs=epochs)

# Vocabulario documentos
model.build_vocab(documents)

# Treinamentos modelo
model.train(documents,
            total_examples=model.corpus_count,
            epochs=model.epochs)

In [ ]:
# Exemplo de uso do modelo
example_text = "This is an example text for prediction"

In [ ]:
# Visualizando modelo
example_tokens = preprocess_text(example_text)
example_tokens

In [ ]:
# Visualizando os vectores
vector = model.infer_vector(example_tokens)
vector

In [ ]:
# Similaridade
similar_documents = model.docvecs.most_similar([vector])
similar_documents

# **Extração de tópicos - LDA**


Extração de tópicos com LDA (Latent Dirichlet Allocation) é uma técnica utilizada no processamento de linguagem natural (NLP) para identificar tópicos ocultos em um conjunto de documentos. O LDA é um modelo probabilístico que atribui palavras a tópicos e tópicos a documentos, permitindo uma análise estruturada e organizada dos textos. O LDA parte do pressuposto de que cada documento é uma combinação de vários tópicos e que cada tópico é uma distribuição de palavras. Ele assume que a geração dos documentos segue um processo probabilístico latente, em que os tópicos são selecionados aleatoriamente e, em seguida, palavras são escolhidas com base nesses tópicos. O processo de extração de tópicos com LDA envolve as seguintes etapas:

A) Pré-processamento: Os documentos são pré-processados para remover stopwords (palavras comuns sem significado específico) e realizar a tokenização, que consiste em separar o texto em unidades menores, como palavras.

B) Representação vetorial: Os documentos são transformados em uma representação vetorial, geralmente utilizando a frequência das palavras (bag-of-words) ou técnicas mais avançadas, como o TF-IDF (Term Frequency-Inverse Document Frequency).

C) Treinamento do modelo: O modelo LDA é treinado no conjunto de documentos representados vetorialmente. Durante o treinamento, o LDA estima a distribuição dos tópicos em cada documento e a distribuição das palavras em cada tópico. Esse processo é realizado de forma iterativa, ajustando os parâmetros do modelo para maximizar a probabilidade de gerar os documentos observados.

D) Inferência de tópicos: Após o treinamento, o modelo LDA pode ser utilizado para inferir os tópicos em novos documentos. Ele atribui probabilidades para cada palavra pertencer a cada tópico, permitindo identificar os tópicos mais relevantes em um documento específico.

A extração de tópicos com LDA é amplamente utilizada em várias aplicações de NLP, como agrupamento de documentos, sumarização automática, análise de sentimentos, recomendação de conteúdo e muito mais. Essa técnica ajuda a organizar grandes volumes de texto, identificando temas subjacentes e facilitando a compreensão e o processamento dos documentos. Ao extrair tópicos com LDA, é possível obter uma visão mais estruturada e organizada dos dados textuais, permitindo insights e análises mais precisas em várias áreas de estudo e aplicação.

In [ ]:
# Carregar os dados
data = pd.read_csv('training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
data.head()

In [ ]:
# Pré-processamento dos dados para LDA
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r'\d+', '', text)  # Remover números
    text = re.sub(r'[^\w\s]', '', text)  # Remover pontuação e caracteres especiais
    tokens = word_tokenize(text)  # Tokenização
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lematização
    tokens = [token for token in tokens if token not in stop_words]  # Remover stopwords
    text = ' '.join(tokens)
    return text

# Visualizando o texto novo
data['texto_preprocessado'] = data['Text'].apply(preprocess_text)
data.head()

In [ ]:
# Vetorização dos dados
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['texto_preprocessado'])
feature_names = vectorizer.get_feature_names_out()  # Correção aqui

In [ ]:
# Aplicação do LDA

# Defina o número de tópicos desejado
num_topics = 10

# Modelo LDA
lda = LatentDirichletAllocation(n_components=num_topics,
                                random_state=42)

# Treinamento modelo lda
lda.fit(X)

In [ ]:
# Obtendo as palavras mais relevantes para cada tópico
for topic_idx, topic in enumerate(lda.components_):
    print(f"Top 10 palavras para o tópico #{topic_idx}:")
    top_words_indices = topic.argsort()[:-11:-1]
    top_words = [feature_names[i] for i in top_words_indices]
    print(top_words)

In [ ]:
# Passo 5: Atribuição de tópicos aos documentos
topic_assignments = lda.transform(X)
data['topicos'] = topic_assignments.argmax(axis=1)

In [ ]:
# Exemplo de análise dos resultados
print(data[['Text', 'topicos']].head(8))  # Exibe os 10 primeiros documentos e seus tópicos atribuídos

In [ ]:
# Criar um novo DataFrame com os resultados dos tópicos
resultados = pd.DataFrame({'Text': data['Text'], 'topicos': data['topicos']})
resultados.head()

In [ ]:
# Passo 5: Obter as palavras mais relevantes para cada tópico
topic_words = []
for topic_idx, topic in enumerate(lda.components_):
    top_words_indices = topic.argsort()[:-11:-1]
    top_words = [feature_names[i] for i in top_words_indices]
    topic_words.append(top_words)

In [ ]:
# Visualizar os tópicos com WordCloud
for i, topic_words in enumerate(topic_words):
    wordcloud = WordCloud(width=800,
                          height=400,
                          background_color='white').generate(' '.join(topic_words))
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Tópico #{i}')
    plt.axis('off')
    plt.show()

In [ ]:
# Salvar o DataFrame em um arquivo CSV
resultados.to_csv('resultados_topicos.csv', index=False)

# **Pré-processamento - ANN**

Pré-processamento é uma etapa crucial no processamento de dados, incluindo o processamento de linguagem natural (NLP). É um conjunto de técnicas e procedimentos aplicados aos dados brutos com o objetivo de prepará-los para uma análise mais eficiente e precisa. No contexto do processamento de linguagem natural, o pré-processamento refere-se ao conjunto de transformações aplicadas ao texto antes de ser utilizado em algoritmos ou modelos de aprendizado de máquina. Essas transformações visam limpar, normalizar e estruturar o texto, removendo informações desnecessárias, padronizando formatos e preparando os dados para uma análise mais significativa. Algumas técnicas comuns de pré-processamento incluem:

A) Tokenização: Consiste em dividir o texto em unidades menores, chamadas tokens. Esses tokens geralmente são palavras individuais, mas também podem ser frases ou outros elementos, dependendo do objetivo da análise.

B) Remoção de stopwords: Stopwords são palavras comuns, como "o", "é", "de", que ocorrem com frequência, mas geralmente não contribuem significativamente para o significado do texto. A remoção dessas palavras pode reduzir o ruído e o tamanho do vocabulário.

C) Remoção de pontuação e caracteres especiais: Pontuação, símbolos e outros caracteres especiais podem ser removidos, a menos que tenham um significado específico no contexto da análise.

D) Normalização de palavras: Isso envolve a aplicação de técnicas como a lematização (redução de palavras à sua forma base ou lema) e a stemming (redução de palavras à sua raiz) para reduzir a variação morfológica das palavras e agrupar variações semelhantes.

E) Remoção de números e dígitos: Se os números não são relevantes para a análise em questão, eles podem ser removidos para simplificar o texto.

F) Normalização de caixa: Pode-se converter todas as letras para maiúsculas ou minúsculas, a fim de evitar duplicações desnecessárias de palavras devido à diferenciação de caixa.

Essas são apenas algumas técnicas de pré-processamento comuns, e a escolha de quais aplicar depende do objetivo da análise e do tipo de texto sendo processado. O pré-processamento adequado é essencial para melhorar a qualidade dos dados, remover ruídos e garantir que o texto esteja em uma forma mais adequada para análise posterior, como classificação de texto, agrupamento ou extração de informações.

In [ ]:
import os
from gensim.models import KeyedVectors

model_word2vec = None
if (model_word2vec == None):
    print("word2vec não carregado!")
else:
    print("Encontrados {} vetores de palavras de word2vec".format(len(model_word2vec.vocab)))

In [ ]:
# definido para o valor máximo do conjunto de dados de origem
size = len(train)

# verifique a distribuição de classes e encontre o tamanho mínimo da amostra
for x in range(1,20):
  _= len(train[train["Class"] == x])
  if (_ <size):
    size =_

# Novo dataframe
train_x = pd.DataFrame(columns = train.columns)

for y in range(1, 20):

  # Adicionando nova lista
  train_x = train_x.append(train[train['Class']==y][:size], ignore_index=True)

def data_tra(l, n):
  for i in range(0, len(l), n):
    yield l[i:i+n]

sets = 200
setd = 3

base = {'Text': [], 'Class': [], 'ID': [], 'Gene': [], 'Variation': []}
for id, row in train.iterrows():
  toke_list = nltk.sent_tokenize(row["Text"])
  if (len(toke_list) > setd):
    toke_list = toke_list[len(toke_list)-sets:]
  set_data = list(data_tra(toke_list, setd))

  for data2 in set_data:
    base['Text'].append(" ".join(data2))
    base['Class'].append(row['Class'])
    base['ID'].append(row['ID'])
    base['Gene'].append(row['Gene'])
    base['Variation'].append(row['Variation'])

# Novo dados treinamento
datalen = len(train)
train = pd.DataFrame(base)
train

# **Tokenização das palavras**

Tokenização é o processo de dividir um texto em unidades menores chamadas tokens. Esses tokens podem ser palavras individuais, frases, símbolos ou qualquer outra unidade significativa, dependendo do contexto e dos requisitos da análise. A tokenização é uma etapa fundamental no processamento de linguagem natural (NLP) e serve como base para várias tarefas, como classificação de texto, análise de sentimento, tradução automática, entre outras. Ao dividir o texto em tokens, torna-se possível analisar e extrair informações de forma mais granular e eficiente. Aqui estão alguns pontos importantes sobre a tokenização:

A) Unidades de token: Os tokens podem ser definidos de diferentes maneiras, dependendo do contexto. A unidade mais comum é a palavra individual. No entanto, em algumas situações, pode ser necessário considerar frases, parágrafos ou até mesmo caracteres individuais como tokens.

B) Processo de divisão: A tokenização geralmente envolve a divisão do texto com base em delimitadores, como espaços em branco, pontuação ou outros caracteres específicos. Por exemplo, a frase "Eu gosto de programação" pode ser tokenizada em quatro palavras: "Eu", "gosto", "de" e "programação".

C) Normalização: Em muitos casos, os tokens também são normalizados durante o processo de tokenização. Isso pode incluir a conversão de todas as letras para minúsculas, remoção de acentos ou outras formas de normalização linguística.

D) Tratamento de casos especiais: Alguns casos especiais podem surgir durante a tokenização, como contrações ("não" + "é" = "não é"), símbolos ou acrônimos. O tratamento adequado desses casos depende das necessidades da análise e pode exigir regras específicas ou modelos de aprendizado de máquina.

A tokenização é uma etapa essencial no pré-processamento de texto em NLP, pois fornece a base para a análise subsequente. Ao dividir o texto em unidades menores, os dados são estruturados de forma mais adequada para análises mais avançadas, permitindo que os algoritmos compreendam e processem o texto de maneira mais eficiente.

In [ ]:
# Máximo de palavras
n_words = 5000

# Comprimento máximo da sequência
s_length = 200

# Tokenizer
token = Tokenizer(num_words = n_words)

# Treinamento
token_fit = token.fit_on_texts(train['Text'])

# Obter sequências
X = token.texts_to_sequences(train['Text'])

# Palavras únicas no texto
word_index = token.word_index
print("Found {} unique tokens.".format(len(word_index)))

# Sequências de pad
X = pad_sequences(X, maxlen=s_length)

# Embedding text
emb = None

# Modelo word2vec
if (model_word2vec != None):
  words = []
  emb = np.zeros((num_words+1, 200)) #200 = word2vec dim
  for x in word_index.items():
    if x > n_words:
      continue
    if x in model_word2vec.vocab:
      emb[i] = model_word2vec(x)
    else:
      words.append(x)

# **Rede neural**


Uma rede neural para processamento de linguagem natural (NLP) é um tipo de modelo de aprendizado de máquina que é projetado para lidar especificamente com tarefas relacionadas ao processamento e compreensão de linguagem humana. Essas redes neurais são treinadas em grandes conjuntos de dados textuais para aprender padrões, relações semânticas e sintáticas, e realizar tarefas específicas relacionadas à linguagem. Existem várias arquiteturas de redes neurais usadas no processamento de linguagem natural, cada uma com seus próprios pontos fortes e aplicações. Aqui estão algumas das arquiteturas mais comuns:

A) Redes Neurais Recorrentes (RNN): As RNNs são adequadas para lidar com sequências de dados, como frases ou documentos. Elas possuem conexões recursivas que permitem que a informação seja passada de um passo de tempo para outro, capturando assim o contexto e a dependência entre as palavras.

B) Long Short-Term Memory (LSTM): As LSTMs são uma variação das RNNs, projetadas para superar o problema de dependências de longo prazo. Elas têm uma estrutura de memória interna que permite reter informações relevantes por períodos mais longos, sendo úteis para tarefas de linguagem com dependências de longa distância, como tradução automática ou geração de texto.

C) Redes Neurais Convolucionais (CNN): As CNNs, originalmente desenvolvidas para visão computacional, também são aplicadas com sucesso em NLP. Elas podem ser usadas para extrair recursos locais, como n-gramas, de sequências de palavras, sendo úteis para tarefas como classificação de texto e análise de sentimentos.

D) Transformers: Os Transformers são uma arquitetura mais recente e altamente popular no processamento de linguagem natural. Eles se destacam por sua capacidade de capturar relações de dependência entre palavras, eliminando a necessidade de conexões recorrentes. Os Transformers são amplamente utilizados para tarefas como tradução automática, resumo de texto e geração de texto.

Essas são apenas algumas das arquiteturas de redes neurais usadas no processamento de linguagem natural. Cada uma delas possui características distintas e é aplicada de acordo com a tarefa específica e os requisitos do problema em questão. As redes neurais para processamento de linguagem natural têm sido fundamentais para avanços significativos em áreas como tradução automática, análise de sentimentos, processamento de fala, chatbots e muito mais.

In [ ]:
# Rede neural

# Word2vec entrada
dim = 200

# Modelo
model_file_name = 'model'

# Dados treino
Y = train['Class'].values-1

# One hot com dados coluna target
Y = keras.utils.to_categorical(Y)

# Aumento do tamanho do lote no ambiente local
batch_size = 20

# Épocas da rede
epochs = 100

# salvamento de modelo com callback
ckpt_callback = keras.callbacks.ModelCheckpoint(model_file_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

# input layer
input1 = keras.layers.Input(shape=(s_length,))

# camada de incorporação
if (emb == None):
    # word2vec não foi carregado. usar método alternativo
    embedding = keras.layers.Embedding(n_words+1, dim, trainable=True)(input1)
else:
    # word2vec foi carregado, pesos de carga e definido como não treinável
    embedding = keras.layers.Embedding(n_words+1, dim, weights=[emb], trainable=False)(input1)

# camadas de conversão ANN
convs = []
filter_sizes = [2,3,4]
for fsz in filter_sizes:
    l_conv = keras.layers.Conv1D(filters=100,kernel_size=fsz,activation='relu')(embedding)
    l_pool = keras.layers.MaxPooling1D(s_length-100+1)(l_conv)
    l_pool = keras.layers.Flatten()(l_pool)
    convs.append(l_pool)

# ANN
l_merge = keras.layers.concatenate(convs, axis=1)
l_out = keras.layers.Dropout(0.5)(l_merge)
output = keras.layers.Dense(units=9, activation='softmax')(l_out)
model = keras.models.Model(input1, output)

# compile model
ANN_model = model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['categorical_crossentropy'])

# Summary ANN
model.summary()

In [ ]:
# Treinamento modelo
ANN_model = model.fit(X,
                      Y,
                      epochs=epochs,
                      batch_size=batch_size,
                      validation_split=0.2,
                      verbose=1)

In [ ]:
print("Training accuracy: %.2f%% / Validation accuracy: %.2f%%" % (100*ANN_model.history['categorical_crossentropy'][-1], 100*ANN_model.history['val_categorical_crossentropy'][-1]))

In [ ]:
# summarize history
plt.plot(ANN_model.history['val_loss'])
plt.plot(ANN_model.history['val_categorical_crossentropy'])
plt.title('Precisão do modelo')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# summarize history loss
plt.plot(ANN_model.history['loss'])
plt.plot(ANN_model.history['val_loss'])
plt.title('Perda do modelo')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# **Dados treino ANN**

In [ ]:
# Dados teste
data_test = pd.read_csv('/content/training_variants')

# Carregando dados texto
test_txt_ = pd.read_csv('stage2_test_text.csv',
                        sep='\|\|',
                        engine='python',
                        header=None,
                        skiprows=1,
                        names=["ID","Text"])

# Carregando csv dados teste
test_data = pd.merge(data_test,
                     test_txt_,
                     how='left',
                     on='ID')

In [ ]:
tmpdf_ = {'Text': [], 'ID': [], 'Gene': [], 'Variation': []}

for index, row in test.iterrows():
    toke = nltk.sent_tokenize(row['Text'])
    if (len(toke) > sets):
        toke = toke[len(toke)-sets:]
    data_r = list(data_tra(toke, setd))
    for chunk in data_r:
      tmpdf_['Text'].append(" ".join(chunk))
      tmpdf_['ID'].append(row['ID'])
      tmpdf_['Gene'].append(row['Gene'])
      tmpdf_['Variation'].append(row['Variation'])

# create new train set from temp dict
origtestlen = len(test)
test = pd.DataFrame(tmpdf_)

# clean up
del tmpdf_

# Visualizando
test

In [ ]:
# get sequences
Xtest = token.texts_to_sequences(test['Text'])

In [ ]:
# pad sequences
Xtest = pad_sequences(Xtest, maxlen=s_length)

In [ ]:
# Previsão rede neural
pred = model.predict(Xtest, verbose=1)
print(pred)

In [ ]:
# Resultado final ANN salvando csv
data_final = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])

# Incluindo ids
data_final.insert(loc=0, column='ID', value=test['ID'].values)

# Media resultados
data_final = data_final.groupby(['ID'], as_index=False).mean()

In [ ]:
# Visualizando os resultados
data_final

In [ ]:
# Salvando em csv dataset
data_final.to_csv('dataset_resultados_final.csv', index=False)

# **Conclusão**


Neste projeto, desenvolvi uma abordagem utilizando redes neurais para classificar variações genéticas. Para enriquecer a representação dos dados, utilizei modelos de processamento de linguagem natural, como Word2Vec e Doc2Vec, para extrair informações relevantes das palavras-chave presentes nos textos. Esses modelos permitiram capturar relações semânticas e contextuais, contribuindo para uma melhor compreensão das variações genéticas.

Além disso, como uma etapa adicional, implementei um modelo de extração de tópicos usando o LDA (Latent Dirichlet Allocation). Essa técnica identifica tópicos latentes presentes nos dados e auxilia na compreensão global do corpus. Ao final deste processo, utilizei um gráfico de nuvem de palavras para visualizar as palavras-chave mais relevantes em cada tópico, oferecendo uma representação visual clara e concisa.

Por fim, implementei um terceiro modelo baseado em rede neural, que foi treinado para classificar as variações genéticas. Esse modelo foi projetado para aprender padrões complexos e realizar uma classificação precisa. A rede neural recebeu como entrada as informações extraídas dos modelos anteriores (Word2Vec, Doc2Vec e LDA), o que proporcionou um conjunto de características mais abrangente para a classificação.

O processo de desenvolvimento envolveu a coleta e preparação dos dados genéticos, incluindo a limpeza e padronização dos textos. Em seguida, os modelos de processamento de linguagem natural foram aplicados para extrair informações das palavras-chave e identificar os tópicos relevantes. Essas informações foram utilizadas como entrada para o modelo de rede neural, que foi treinado utilizando técnicas avançadas de aprendizado profundo.

Durante o treinamento do modelo de rede neural, foram aplicadas técnicas de validação cruzada e otimização para garantir que o modelo estivesse bem ajustado aos dados e fosse capaz de realizar uma classificação precisa das variações genéticas.

Em resumo, este projeto empregou uma abordagem multifacetada, utilizando modelos de processamento de linguagem natural (Word2Vec, Doc2Vec e LDA) e redes neurais para a classificação de variações genéticas. Essa combinação de técnicas permitiu uma representação mais rica e abrangente dos dados, contribuindo para resultados mais precisos e confiáveis.


# **Referência**

Link 1 - https://medium.com/@everton.tomalok/word2vec-e-sua-import%C3%A2ncia-na-etapa-de-pr%C3%A9-processamento-d0813acfc8ab

Link 2 - https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

Link 3 - https://www.kaggle.com/code/umutto/preliminary-data-analysis-using-word2vec/notebook

Link 4 - https://www.kaggle.com/code/bostjanm/cnn-aproach-to-text-mining

Link 5 - https://www.kaggle.com/competitions/msk-redefining-cancer-treatment

# **Citação**

Esse projeto da formação cientista de dados pela DSA.

@Data Science Academy

https://www.datascienceacademy.com.br/